<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Manipulación de Datos con la librería Pandas II


<img style="float: right; margin: 0px 0px 15px 15px;" src="https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png" width="400px" height="400px" />

> La clase pasada comenzamos a ver las funcionalidades de la librería [pandas](https://pandas.pydata.org/). Básicamente, aprendimos a cargar datos desde archivos `.csv`, aprendimos a seleccionar subcojuntos de estos datos mediante indización (obtener ciertas filas, o ciertas columnas, o ciertas filas y columnas determinadas). Finalmente, aprendimos a filtrar datos, es decir, a seleccionar registros de la base de datos que satisfagan cierta condición.

> Hoy nos enfocaremos a ver como reunir varias bases de datos que poseen información complementaria relativa a un mismo problema, para obtener una sola tabla con la información condensada.

Referencias:
- https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
___

# 0. Motivación

En cualquier situación relacionada con ciencia de datos en la vida real, no pasarán más de 10 minutos sin que aparezca la necesidad de unir (`merge` o `join`) dos bases de datos complementarias para formar tu conjunto de datos para el análisis.

La unión de DataFrames es un proceso fundamental, que cualquier analista de datos en formación debe aprender. En esta clase aprenderemos cómo hacerlo, y en el camino responderemos las siguientes preguntas:

- ¿Qué son el `merge` o `join` de dos DataFrames?

- ¿Qué son `inner`, `outer`, `left` y `right` `joins`?



## Datos para la clase

En esta clase, seguiremos trabajando con los datos de la clase pasada:

En la carpeta "data" tenemos los archivos 

- "customers_data.csv": datos propios de los clientes de la empresa, 
- "sessions_data.csv": inicios de sesión de los clientes en la plataforma web de compras (similar a amazon),
- "transactions_data.csv": transacciones asociadas a cada inicio de sesión, y
- "products_data.csv": información de los productos. 

Podemos cargar cada uno de los archivos `.csv` como DataFrames de Pandas usando la función `read_csv()` de pandas, y examinar los contenidos de cada uno con el método `head()`.

In [2]:
# Importar pandas
import pandas as pd
import numpy as np

In [23]:
# Cargar customers_data
customers = pd.read_csv('Data/customers_data.csv', index_col=0)
customers.head(5)

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [6]:
customers.shape

(5, 4)

In [7]:
# Cargar sessions_data
sesiones = pd.read_csv('Data/sessions_data.csv', index_col=0)
sesiones.head(5)

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [8]:
sesiones.shape

(35, 4)

In [9]:
sesiones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     35 non-null     int64 
 1   customer_id    35 non-null     int64 
 2   device         35 non-null     object
 3   session_start  35 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [10]:
# Cargar transactions_data
transacciones = pd.read_csv('Data/transactions_data.csv', index_col=0)
transacciones.head(5)

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [11]:
transacciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    500 non-null    int64  
 1   session_id        500 non-null    int64  
 2   transaction_time  500 non-null    object 
 3   product_id        500 non-null    int64  
 4   amount            500 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 23.4+ KB


In [12]:
# Cargar products_data
products = pd.read_csv('Data/products_data.csv', index_col=0)
products.head(5)

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [13]:
products.shape

(5, 2)

## Bonus: formato de fechas y horas

Como vimos la vez pasada, es relevante en general un correcto manejo de las fechas y horas en las bases de datos.

Lo primero que se tiene que hacer para un correcto manejo de las fechas (y horas) es identificar las columnas o variables que contienen fechas. Por ejemplo, de la tabla `customers_data`:

In [22]:
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,NaT,1994-07-18
1,2,13244,NaT,1986-08-18
2,3,13244,NaT,2003-11-21
3,4,60091,NaT,2006-08-15
4,5,60091,NaT,1984-07-28


Lo que sigue, es especificarle a pandas el formato en que vienen esas fechas con la función `to_datetime()` de pandas

In [17]:
# Ayuda en la función to_datetime
help(pd.to_datetime)

Help on function to_datetime in module pandas.core.tools.datetimes:

to_datetime(
    arg: 'DatetimeScalarOrArrayConvertible | DictConvertible',
    errors: 'DateTimeErrorChoices' = 'raise',
    dayfirst: 'bool' = False,
    yearfirst: 'bool' = False,
    utc: 'bool' = False,
    format: 'str | None' = None,
    exact: 'bool | lib.NoDefault' = <no_default>,
    unit: 'str | None' = None,
    infer_datetime_format: 'lib.NoDefault | bool' = <no_default>,
    origin: 'str' = 'unix',
    cache: 'bool' = True
) -> 'DatetimeIndex | Series | DatetimeScalar | NaTType | None'
    Convert argument to datetime.

    This function converts a scalar, array-like, :class:`Series` or
    :class:`DataFrame`/dict-like to a pandas datetime object.

    Parameters
    ----------
    arg : int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like
        The object to convert to a datetime. If a :class:`DataFrame` is provided, the
        method expects minimally the following columns:

In [ ]:
# Especificar el formato de fechas en la tabla customers_data
pd.to_datetime(customers['date_of_birth'],format="%Y-%m-%d", errors='coerce')

Hagan esto mismo para todas las tablas:

In [24]:
customers['join_date']=pd.to_datetime(customers['join_date'],format="%Y-%m-%d %H:%M:%S", errors='coerce')
customers['date_of_birth']=pd.to_datetime(customers['date_of_birth'],format="%Y-%m-%d", errors='coerce')

customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [ ]:
# Especificar el formato de fechas en la tabla sessions


In [ ]:
# Especificar el formato de fechas en la tabla customers


## Volviendo al problema ...

Hay columnas o variables que relacionan las tablas. Por ejemplo:

- Los clientes pueden iniciar sesión en la plataforma cuantas veces quieran. De manera que hay una relación uno a muchos entre las tablas "customers_data" y "sessions_data", mediante la variable "customer_id".

- Cuando se efectúa una transacción se supone que se está comprando un producto. Por lo tanto hay una relación uno a muchos entre las tablas "transactions_data"  y "products_data", mediante la variable "product_id".

## Problemas

Primero, quisieramos determinar cuál es el dispositivo preferido por zona.

Luego, quisiéramos determinar cuál es la marca preferida por zona.

Para resolver lo anterior necesitamos unir ("merge" o "join") nuestros datasets en uno solo para el análisis.

___
# 1. Uniendo dos DataFrames...

> “Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

Las palabras “merge” y “join” se usan indistintamente en Pandas, y en otros lenguajes como SQL y R. En Pandas, hay métodos separados “merge” y “join”, que realizan cosas similares (personalmente uso el método "merge"), y la función "merge".

Vamos a concentrarnos en el **primer problema**. En este escenario, necesitamos llevar a cabo los siguientes pasos:

- Para cada fila en el dataset `sessions_data`, debemos hacer una nueva columna que contenga el "zip_code" respectivo de cada cliente.
- Una vez hagamos esto, obtenemos la moda de los dispositivos para cada cliente.

**¿Podemos usar un ciclo?**

Claro que si. Se podría escribir un ciclo para esta tarea. Éste iría a través de cada fila en `sessions_data`, y a cada "user_id" asignar el valor de la nueva columna con la zona respectiva.

Sin embargo, usar ciclos haría nuestra tarea mucho más lenta y plagada de más código que el necesario que si se usara la función (método) `join()`.

De forma que, para estas situaciones, **nunca usar ciclos**.

Para ver cómo podemos hacer lo anterior, veamos la ayuda de la función `merge()` de pandas:

In [29]:
# Ayuda de merge()
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


Ahora, veamos como podemos añadir el código postal a la tabla `sessions_data`, usando la función `merge()` de pandas:

In [38]:
# Uso de la función merge()
sessions_with_zip = pd.merge(left=sesiones,
                              right=customers[['customer_id', 'zip_code']],
                              on="customer_id")
#sesiones.head(5)
sessions_with_zip.head(5)

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,2,5,mobile,2014-01-01 00:17:20,60091
2,3,4,mobile,2014-01-01 00:28:10,60091
3,4,1,mobile,2014-01-01 00:44:25,60091
4,5,4,mobile,2014-01-01 01:11:30,60091


La función `merge()` es el objetivo principal de esta clase. Básicamente, la operación de unir dos DataFrames hace lo siguiente: 

- toma el DataFrame de la izquierda (argumento left=), 
- el DataFrame de la derecha (argumento right=),
- la columna donde se va a unir (argumento on=), y
- la forma en que se va a unir (argumento how=).

La variable en común sobre la que se hace la unión está especificada en el argumento `on`.

Con este resultado, podemos filtrar por zona y luego obtener la moda.

In [ ]:
# Obtener la moda por zona


### Usando GroupBy

### ¿Qué son los tipos de unión inner, left, right y outer?

En nuestro ejemplo, unimos `sessions` con `customers` sobre la columna "custumers_id". En este caso, en ambas tablas existían todos los posibles valores de "customers_id" (1, 2, 3, 4, 5).

¿Qué pasa si esta situación no se cumple?

Por ejemplo, realicemos los siguientes cambios artificiales:

In [ ]:
# artificial sessions_data

# artificial customers_data


Por defecto, la operación `merge()` de pandas actpua con un merge tipo "inner". Un "inner merge", guarda únicamente los valores comúnes (en la columna especificada en el argumento `on=`) de ambos DataFrames.

Por ejemplo:

In [ ]:
art_sessions

In [ ]:
# inner join


Los otros comportamientos se pueden revisar mejor a través de ejemplos:

In [ ]:
# left join


In [ ]:
# right join


In [ ]:
# outer join


Estos comportamientos son muy intuitivos a partir de sus valores.

Así mismo, sus usos son muy intuitivos (lo sabrán cuando se enfrenten a problemas reales).

### Resolvamos el problema 2

Determinar cuál es la marca preferida por zona.